In [25]:
# Dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

In [4]:
# Read in csv and review dataframe
file_path = 'Resources/lending_data.csv'
df_credit = pd.read_csv(file_path)
df_credit.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [12]:
# Seperate features and labels set from 'loan_status' column
X = df_credit.copy()
X = X.drop(columns= ['loan_status'])

# Define the target
y = df_credit['loan_status']

In [20]:
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [13]:
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


In [15]:
# Check the balance of the labels
print(y.value_counts())

0    75036
1     2500
Name: loan_status, dtype: int64


In [21]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [22]:
# Instatiate linear model and fit the model to the training data
linear_model = LogisticRegression(solver='lbfgs', random_state=1)

linear_model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [23]:
# Predictions using the testing data
predictions = linear_model.predict(X_test)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
# Calculate accuracy score
balanced_accuracy_score(y_test, predictions)

0.9481182566723452

In [27]:
# Confusion matrix
confusion_matrix(y_test, predictions)

array([[18682,   102],
       [   59,   541]])

In [28]:
# Classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18784
           1       0.84      0.90      0.87       600

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.93     19384
weighted avg       0.99      0.99      0.99     19384

